# 2024 Election Ongoing Statistics

## Import Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from datetime import datetime
from re import search
from os import listdir
from pandas import DataFrame
from warnings import filterwarnings
from threading import Thread, Lock
filterwarnings('ignore')

## Supporting Functions

In [2]:
def refresh_country_stats(driver: webdriver, state_names: list, results: dict, electoral_votes: dict):
    driver.refresh()
    while True:
        try:
            show_more_button = driver.find_element(By.CLASS_NAME, "house-nationalsstyles__FullRaceDetails-sc-10qfp1h-7")
            # show_more_button.click()
            driver.execute_script("arguments[0].click();", show_more_button)
            sleep(1)  # wait for the content to load
        except Exception as e:
            break
    
    page = driver.find_element(By.XPATH, "/html/body").text
    extraction_time = datetime.now()
    if(extraction_time.month>=11 and extraction_time.day>=10 and extraction_time.hour>=8 and extraction_time.minute>=0):
        return
    lines = page.split('\n')
    del page
    state_results_found = False
    current_state = 'None'
    current_state_found = False
    current_state_leading_candidate = 'None'
    current_state_leading_candidate_vote_percent = 0
    current_state_leading_candidate_vote_count = 0
    current_state_trailing_candidate = 'None'
    current_state_trailing_candidate_vote_percent = 0
    current_state_trailing_candidate_vote_count = 0
    current_state_reported_percent = 0
    current_state_electoral_votes = 0
    for line in range(len(lines)):
        if(not(state_results_found)):
            if('STATE RESULTS' in lines[line]):
                state_results_found = True
        if(state_results_found):
            if(not(current_state_found)):
                for state in state_names:
                    if(state in lines[line]):
                        current_state = state
                        current_state_electoral_votes = electoral_votes[state]
                        current_state_found = True
            if('Candidate % Votes' in lines[line]):
                current_state_leading_candidate = lines[line+1]
                if('trump' in current_state_leading_candidate.lower()):
                    current_state_trailing_candidate = 'Biden'
                else:
                    current_state_trailing_candidate = 'Trump'
            if(
                (current_state_trailing_candidate_vote_percent==0)and 
                (not(current_state_leading_candidate_vote_percent==0))and  
                (search(r"[0-9]{1,}",lines[line]))and
                ('%' in lines[line])
                ):
                current_state_trailing_candidate_vote_percent = float(lines[line].replace('%',''))
            if(
                (current_state_leading_candidate_vote_percent==0)and 
                (search(r"[0-9]{1,}",lines[line]))and
                ('%' in lines[line])
                ):
                current_state_leading_candidate_vote_percent = float(lines[line].replace('%',''))
            if(
                (current_state_trailing_candidate_vote_count==0)and 
                (not(current_state_leading_candidate_vote_count==0))and 
                (search(r'\d+(?:,\d+)?',lines[line]))and
                (not('%' in lines[line]))and
                (not('.' in lines[line]))and
                (not(':' in lines[line]))and
                (not('Elect' in lines[line]))and
                (not(search(r"[A-Za-z]{1,}",lines[line])))
                ):
                current_state_trailing_candidate_vote_count = int(lines[line].replace(',',''))
            if(
                (current_state_trailing_candidate_vote_count==0)and 
                (current_state_leading_candidate_vote_count==0)and 
                (search(r'\d+(?:,\d+)?',lines[line]))and
                (not('%' in lines[line]))and
                (not('.' in lines[line]))and
                (not(':' in lines[line]))and
                (not('Elect' in lines[line]))and
                (not(search(r"[A-Za-z]{1,}",lines[line])))
                ):
                current_state_leading_candidate_vote_count = int(lines[line].replace(',',''))
            if('Est.'in lines[line]):
                current_state_reported_percent = int(search(r"[0-9]{1,}",lines[line]).group())
                results[current_state].loc[extraction_time] = [current_state,current_state_electoral_votes,
                                                               current_state_leading_candidate,current_state_leading_candidate_vote_percent,
                                                                current_state_leading_candidate_vote_count,current_state_trailing_candidate,
                                                                current_state_trailing_candidate_vote_percent,current_state_trailing_candidate_vote_count,
                                                                current_state_reported_percent]
                if(True):
                    current_state_found = False
                    current_state_leading_candidate = 'None'
                    current_state_leading_candidate_vote_percent = 0
                    current_state_leading_candidate_vote_count = 0
                    current_state_trailing_candidate = 'None'
                    current_state_trailing_candidate_vote_percent = 0
                    current_state_trailing_candidate_vote_count = 0
                    current_state_reported_percent = 0
            if(
                ('Not all candidates are listed' in lines[line])or
                ('All times ET' in lines[line])
            ):
                break
    for key,item in results.items():
        item.to_csv(f"State_Summary_Results/{key}_Results.csv")
    return driver,results

In [3]:
def refresh_individual_state_stats(driver: webdriver, state_names: list, results: dict, electoral_votes: dict, state_site: str, lock: Lock, return_value: dict):
    driver.get(f"https://www.cnn.com/election/2020/results/state/{state_site}/president")
    page = ""
    while True:
        try:
            lines = str(driver.find_element(By.XPATH, "/html/body").text).split('\n')
            final_page = []
            county_results_found = False
            for line in range(len(lines)):
                if(not(county_results_found)):
                    if('county result' in lines[line].lower()):
                        county_results_found = True
                if(county_results_found):
                    if(
                        ('Not all candidates are listed' in lines[line])or
                        ('Not all candidates are listed' in lines[line])or
                        ('Not all candidates are listed' in lines[line])or
                        ('Not all candidates are listed' in lines[line])or
                        ('CNN will broadcast a projected winner only after an extensive review ' in lines[line])
                      ):
                        break
                    else:
                        final_page.append(lines[line])
                    
            page += '\n'.join(final_page)
            show_more_button = driver.find_element(By.XPATH, """//div[@class='paginationstyles__Arrow-sc-58lau4-1 kxnBsH']
                                                            //img[contains(@src,'data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIiB3aWR0aD0iMjciIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAyNyAyNyI+CiAgICA8ZGVmcz4KICAgICAgICA8cGF0aCBpZD0iYS1hcnJvdy1yaWdodCIgZD0iTTkuOTUyIDUuOTk0SDEuMzk1YS41OTUuNTk1IDAgMSAxIDAtMS4xOWg4LjU1N2wtMy4yMTgtMy4xOWEuNTkyLjU5MiAwIDAgMSAwLS44NC42MDYuNjA2IDAgMCAxIC44NDkgMGw0LjI0MiA0LjIwNWEuNTkzLjU5MyAwIDAgMSAwIC44NDJsLTQuMjQyIDQuMjA1YS42MDYuNjA2IDAgMCAxLS44NSAwIC41OTMuNTkzIDAgMCAxIDAtLjg0MWwzLjIyLTMuMTl6Ii8+CiAgICA8L2RlZnM+CiAgICA8ZyBmaWxsPSJub25lIiBmaWxsLXJ1bGU9ImV2ZW5vZGQiIHRyYW5zZm9ybT0idHJhbnNsYXRlKDEgMSkiPgogICAgICAgIDx1c2UgZmlsbD0iIzRENEQ0RCIgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoNSA3KSIgeGxpbms6aHJlZj0iI2EtYXJyb3ctcmlnaHQiLz4KICAgICAgICA8Y2lyY2xlIGN4PSIxMi41IiBjeT0iMTIuNSIgcj0iMTIuNSIgc3Ryb2tlPSIjNEQ0RDREIi8+CiAgICA8L2c+Cjwvc3ZnPgo=')]""")
            show_more_button.click()
            # driver.execute_script("arguments[0].click();", show_more_button)
            sleep(1)  # wait for the content to load
        except Exception as e:
            break
    
    if(state_site):
        with lock:
            return_value[state_site] = page
            return return_value
    extraction_time = datetime.now()
    if(extraction_time.month>=11 and extraction_time.day>=19 and extraction_time.hour>=8 ):
        return
    lines = page.split('\n')
    del page
    state_results_found = False
    current_state = 'None'
    current_state_found = False
    current_state_leading_candidate = 'None'
    current_state_leading_candidate_vote_percent = 0
    current_state_leading_candidate_vote_count = 0
    current_state_trailing_candidate = 'None'
    current_state_trailing_candidate_vote_percent = 0
    current_state_trailing_candidate_vote_count = 0
    current_state_reported_percent = 0
    current_state_electoral_votes = 0
    for line in range(len(lines)):
        if(not(state_results_found)):
            if('STATE RESULTS' in lines[line]):
                state_results_found = True
        if(state_results_found):
            if(not(current_state_found)):
                for state in state_names:
                    if(state in lines[line]):
                        current_state = state
                        current_state_electoral_votes = electoral_votes[state]
                        current_state_found = True
            if('Candidate % Votes' in lines[line]):
                current_state_leading_candidate = lines[line+1]
                if('trump' in current_state_leading_candidate.lower()):
                    current_state_trailing_candidate = 'Biden'
                else:
                    current_state_trailing_candidate = 'Trump'
            if(
                (current_state_trailing_candidate_vote_percent==0)and 
                (not(current_state_leading_candidate_vote_percent==0))and  
                (search(r"[0-9]{1,}",lines[line]))and
                ('%' in lines[line])
                ):
                current_state_trailing_candidate_vote_percent = float(lines[line].replace('%',''))
            if(
                (current_state_leading_candidate_vote_percent==0)and 
                (search(r"[0-9]{1,}",lines[line]))and
                ('%' in lines[line])
                ):
                current_state_leading_candidate_vote_percent = float(lines[line].replace('%',''))
            if(
                (current_state_trailing_candidate_vote_count==0)and 
                (not(current_state_leading_candidate_vote_count==0))and 
                (search(r'\d+(?:,\d+)?',lines[line]))and
                (not('%' in lines[line]))and
                (not('.' in lines[line]))and
                (not(':' in lines[line]))and
                (not('Elect' in lines[line]))and
                (not(search(r"[A-Za-z]{1,}",lines[line])))
                ):
                current_state_trailing_candidate_vote_count = int(lines[line].replace(',',''))
            if(
                (current_state_trailing_candidate_vote_count==0)and 
                (current_state_leading_candidate_vote_count==0)and 
                (search(r'\d+(?:,\d+)?',lines[line]))and
                (not('%' in lines[line]))and
                (not('.' in lines[line]))and
                (not(':' in lines[line]))and
                (not('Elect' in lines[line]))and
                (not(search(r"[A-Za-z]{1,}",lines[line])))
                ):
                current_state_leading_candidate_vote_count = int(lines[line].replace(',',''))
            if('Est.'in lines[line]):
                current_state_reported_percent = int(search(r"[0-9]{1,}",lines[line]).group())
                results[current_state].loc[extraction_time] = [current_state,current_state_electoral_votes,
                                                               current_state_leading_candidate,current_state_leading_candidate_vote_percent,
                                                                current_state_leading_candidate_vote_count,current_state_trailing_candidate,
                                                                current_state_trailing_candidate_vote_percent,current_state_trailing_candidate_vote_count,
                                                                current_state_reported_percent]
                if(True):
                    current_state_found = False
                    current_state_leading_candidate = 'None'
                    current_state_leading_candidate_vote_percent = 0
                    current_state_leading_candidate_vote_count = 0
                    current_state_trailing_candidate = 'None'
                    current_state_trailing_candidate_vote_percent = 0
                    current_state_trailing_candidate_vote_count = 0
                    current_state_reported_percent = 0
            if(
                ('Not all candidates are listed' in lines[line])or
                ('All times ET' in lines[line])
            ):
                break
    driver.close()
    return results

In [4]:
def exponentiation(x: int, y: int) -> int:
    return x**y

## Initiate Global Variables

In [5]:
# WebDriver Chrome
driver = webdriver.Chrome()
#state_driver = webdriver.Chrome()

In [6]:
state_names = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut",
               "District Of Columbia", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois",
               "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
               "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", 
               "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", 
               "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", 
               "Washington", "Wisconsin", "West Virginia", "Wyoming"]

In [7]:
electoral_votes = {
    "Alabama": 9,
    "Alaska": 3,
    "Arizona": 11,
    "Arkansas": 6,
    "California": 54,
    "Colorado": 10,
    "Connecticut": 7,
    "Delaware": 3,
    "District Of Columbia": 3,
    "Florida": 30,
    "Georgia": 16,
    "Hawaii": 4,
    "Idaho": 4,
    "Illinois": 19,
    "Indiana": 11,
    "Iowa": 6,
    "Kansas": 6,
    "Kentucky": 8,
    "Louisiana": 8,
    "Maine": 4,
    "Maryland": 10,
    "Massachusetts": 11,
    "Michigan": 15,
    "Minnesota": 10,
    "Mississippi": 6,
    "Missouri": 10,
    "Montana": 4,
    "Nebraska": 5,
    "Nevada": 6,
    "New Hampshire": 4,
    "New Jersey": 14,
    "New Mexico": 5,
    "New York": 28,
    "North Carolina": 16,
    "North Dakota": 3,
    "Ohio": 17,
    "Oklahoma": 7,
    "Oregon": 8,
    "Pennsylvania": 19,
    "Rhode Island": 4,
    "South Carolina": 9,
    "South Dakota": 3,
    "Tennessee": 11,
    "Texas": 40,
    "Utah": 6,
    "Vermont": 3,
    "Virginia": 13,
    "Washington": 12,
    "West Virginia": 4,
    "Wisconsin": 10,
    "Wyoming": 3
}

In [8]:
results:dict[str,DataFrame] = {}
for state in state_names:
    results[state] = DataFrame(data=None,columns=['State','Electoral_Votes','Leading_Candidate','Vote_Percent_Leader','Vote_Count_Leader','Trailing_Candidate',
                                                  'Vote_Percent_Trailer','Vote_Count_Trailer','Reported_Percent'])

## Open Summary Webpage

In [9]:
# Target URL
driver.get("https://www.cnn.com/election/2020/results/president")
driver.maximize_window()

## Add Current Summary Data of States to Results

In [10]:
for x in range(10):
    print(x+1)
    driver, results = refresh_country_stats(driver=driver, state_names=state_names, results=results, electoral_votes=electoral_votes)

1
2
3
4
5
6
7
8
9
10


## Create Nationwide Summary Statistics

In [11]:
nation_wide_results = DataFrame(data=None,columns=['Donald_Trump_Current_Vote_Total','Joe_Biden_Current_Vote_Total',
                                                   'Donald_Trump_Current_Vote_Percent','Joe_Biden_Current_Vote_Percent',
                                                   'Donald_Trump_Current_Electoral_Votes','Joe_Biden_Current_Electoral_Votes'
                                                  ])

In [12]:
for key,item in results.items():
    item.to_csv(f"State_Summary_Results/{key}_Results.csv")

In [13]:
state_summary_results = []
donald_trump_vote_totals = []
donald_trump_vote_percents = []
donald_trump_electoral_votes = []
joe_biden_vote_totals = []
joe_biden_vote_percents = []
joe_biden_electoral_votes = []
for key,item in results.items():
    state_summary_results.append(item)
for i in range(0,len(list(state_summary_results[0].index))):
    
    donald_trump_vote_total: int = 0
    donal_trump_electoral_vote: int = 0
    donald_trump_vote_percent: float = 0
    joe_biden_vote_total: int = 0
    joe_biden_electoral_vote: int = 0
    joe_biden_vote_percent: float = 0
    
    for index,state_summary_result in enumerate(state_summary_results):
        if(state_summary_result.loc[[list(state_summary_result.index)[i]]]['Leading_Candidate'][0]=='Trump'):
            donald_trump_vote_total += int(state_summary_result.iloc[[i]]['Vote_Count_Leader'][0])
            donal_trump_electoral_vote += int(state_summary_result.iloc[[i]]['Electoral_Votes'][0])
            joe_biden_vote_total += int(state_summary_result.iloc[[i]]['Vote_Count_Trailer'][0])
        else:
            joe_biden_vote_total += int(state_summary_result.iloc[[i]]['Vote_Count_Leader'][0])
            joe_biden_electoral_vote += int(state_summary_result.iloc[[i]]['Electoral_Votes'][0])
            donald_trump_vote_total += int(state_summary_result.iloc[[i]]['Vote_Count_Trailer'][0])
            
    donald_trump_vote_percent = donald_trump_vote_total/(donald_trump_vote_total+joe_biden_vote_total)
    joe_biden_vote_percent = joe_biden_vote_total/(donald_trump_vote_total+joe_biden_vote_total)
    nation_wide_results.loc[list(state_summary_results[0].index)[i]] = [int(donald_trump_vote_total),int(joe_biden_vote_total),
                                                                        donald_trump_vote_percent,joe_biden_vote_percent,
                                                                        int(donal_trump_electoral_vote),int(joe_biden_electoral_vote)]

In [14]:
nation_wide_results['Donald_Trump_Current_Vote_Total'] = nation_wide_results['Donald_Trump_Current_Vote_Total'].astype(int)
nation_wide_results['Joe_Biden_Current_Vote_Total'] = nation_wide_results['Joe_Biden_Current_Vote_Total'].astype(int)
nation_wide_results['Donald_Trump_Current_Electoral_Votes'] = nation_wide_results['Donald_Trump_Current_Electoral_Votes'].astype(int)
nation_wide_results['Joe_Biden_Current_Electoral_Votes'] = nation_wide_results['Joe_Biden_Current_Electoral_Votes'].astype(int)

In [15]:
nation_wide_results

,Donald_Trump_Current_Vote_Total,Joe_Biden_Current_Vote_Total,Donald_Trump_Current_Vote_Percent,Joe_Biden_Current_Vote_Percent,Donald_Trump_Current_Electoral_Votes,Joe_Biden_Current_Electoral_Votes
2024-08-13 11:02:16.922366,74224319,81284666,0.477299,0.522701,235,303
2024-08-13 11:02:29.708806,74224319,81284666,0.477299,0.522701,235,303
2024-08-13 11:02:42.855530,74224319,81284666,0.477299,0.522701,235,303
2024-08-13 11:02:55.419706,74224319,81284666,0.477299,0.522701,235,303
2024-08-13 11:03:11.296056,74224319,81284666,0.477299,0.522701,235,303
2024-08-13 11:03:24.332159,74224319,81284666,0.477299,0.522701,235,303
2024-08-13 11:03:40.228270,74224319,81284666,0.477299,0.522701,235,303
2024-08-13 11:04:15.887755,74224319,81284666,0.477299,0.522701,235,303
2024-08-13 11:04:30.205869,74224319,81284666,0.477299,0.522701,235,303
2024-08-13 11:04:46.843257,74224319,81284666,0.477299,0.522701,235,303
